In [1]:
import requests
import pprint
import json

import pandas as pd
import numpy as np

# Headers Query String Parameters
# admiCd: 지역코드
# upjongCd: G471903 (편의점)

In [2]:
simpleLoc = ''
admCd = ''
dataUrl = "https://sg.sbiz.or.kr/godo/getAvgAmtInfo.json"
popularUrl = "https://sg.sbiz.or.kr/godo/getPopularInfo.json"

In [3]:
salesAmount = pd.DataFrame()
series = pd.Series()

In [4]:
lines = open('./admCd.txt','r')
for line in lines:
  (simpleLoc, admCd) = line.split(': ')
  if not 'NO' in admCd:
    admCd = line.split('_')[1]

    payload = {'admiCd': int(admCd), 'upjongCd': 'G471903', 'simpleLoc':simpleLoc}
    r = requests.get(dataUrl, params=payload, verify=False)
    # pprint.pprint(json.loads(r.text))
    data = json.loads(r.text)
    annualSales = data['annualSales'] # maxAmt: 최대매출, minAmt: 최소매출, saleAmt: 평균매출 (array)
    series['loc'] = simpleLoc
    for annualSale in annualSales:
        series[annualSale['yymm'] + '_maxAmt' ] = annualSale['maxAmt']
        series[annualSale['yymm'] + '_minAmt' ] = annualSale['minAmt']
        series[annualSale['yymm'] + '_saleAmt'] = annualSale['saleAmt']
    series["guAmt"] = data['guAmt'] # 구 평균
    series["guMax"] = data['guMax'] # 구 max
    series["guMin"] = data['guMin'] # 구 min
    series["prevMonCnt"]  = data['prevMonCnt']  # 이전달 선택업종 업소수
    series["prevYearCnt"] = data['prevYearCnt'] # 이전년 선택업종 업소수
    series["saleCnt"]     = data['saleCnt']     # 현재  선택업종 업소수
    salesAmount = salesAmount.append(series, ignore_index=True)
    print(simpleLoc)

부산 부산진구 부전2동
부산 기장군 기장읍
부산 해운대구 우2동


KeyboardInterrupt: 

In [5]:
salesAmount

,202003_maxAmt,202003_minAmt,202003_saleAmt,202004_maxAmt,202004_minAmt,202004_saleAmt,202005_maxAmt,202005_minAmt,202005_saleAmt,202006_maxAmt,...,202103_maxAmt,202103_minAmt,202103_saleAmt,guAmt,guMax,guMin,loc,prevMonCnt,prevYearCnt,saleCnt
0,13875.0,6213.0,9067.0,14284.0,6397.0,9335.0,15736.0,7047.0,10283.0,15883.0,...,11511.0,2270.0,6702.0,6480,15851,626,부산 부산진구 부전2동,41,45,41
1,8863.0,1329.0,4472.0,8619.0,1292.0,4349.0,9267.0,1389.0,4676.0,9312.0,...,11159.0,1517.0,4770.0,4759,11993,1495,부산 기장군 기장읍,59,47,59
2,7313.0,1600.0,5057.0,6904.0,1510.0,4774.0,7714.0,1688.0,5334.0,7917.0,...,8119.0,2159.0,5658.0,6325,16072,1783,부산 해운대구 우2동,52,48,52
3,11787.0,1790.0,6052.0,11487.0,1744.0,5898.0,12755.0,1937.0,6549.0,12331.0,...,15216.0,2679.0,6914.0,6480,15851,626,부산 부산진구 부전1동,32,31,33
4,9362.0,5835.0,7438.0,9264.0,5774.0,7360.0,9763.0,6085.0,7757.0,9914.0,...,10423.0,3372.0,7723.0,6077,14298,1456,부산 동구 범일2동,12,14,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,6880.0,6880.0,6880.0,6219.0,6219.0,6219.0,7260.0,7260.0,7260.0,6107.0,...,4841.0,4841.0,4841.0,5204,11479,1643,부산 영도구 봉래2동,1,1,1
199,9269.0,5567.0,7418.0,8551.0,5136.0,6844.0,9273.0,5570.0,7421.0,9280.0,...,8394.0,2559.0,6002.0,6103,12319,1305,부산 사상구 모라3동,2,2,2
200,5191.0,1696.0,3688.0,4927.0,1610.0,3500.0,4494.0,1468.0,3193.0,4846.0,...,5793.0,2969.0,4695.0,6077,14298,1456,부산 동구 수정5동,3,2,3
201,11906.0,4874.0,8068.0,11503.0,4708.0,7795.0,11512.0,4712.0,7801.0,12062.0,...,10303.0,4005.0,6802.0,4761,12867,1219,부산 금정구 금사회동동,0,0,0


In [6]:
import datetime

def get_epochtime_ms():
    return round(datetime.datetime.utcnow().timestamp() * 1000)

population = pd.DataFrame()
series2 = pd.Series()

In [7]:
lines = open('./admCd.txt','r')
for line in lines:
  (simpleLoc, admCd) = line.split(': ')
  if not 'NO' in admCd:
    admCd = line.split('_')[1]

    payload = {'admiCd': int(admCd), 'mililis': get_epochtime_ms()}
    r = requests.get(popularUrl, params=payload, verify=False)
    data = (json.loads(r.text))['population']
    series2["loc"]    = simpleLoc # 구 평균
    series2["day"]    = data['day']      # 주중
    series2["weekend"]= data['weekend']  # 주말
    series2["dayAvg"] = data['dayAvg'] # 유동인구 평균
    series2["mon"]    = data['mon']  # 
    series2["tues"]   = data['tues'] # 
    series2["wed"]    = data['wed']  #
    series2["thur"]   = data['thur'] #
    series2["fri"]    = data['fri']  #
    series2["sat"]    = data['sat']  #
    series2["sun"]    = data['sun']  #
    series2["firstHour"]  = data['firstHour']  #  0 ~  3
    series2["secondHour"] = data['secondHour'] #  4 ~  7
    series2["thirdHour"]  = data['thirdHour']  #  8 ~ 11
    series2["fourthHour"] = data['fourthHour'] # 12 ~ 15
    series2["fifthHour"]  = data['fifthHour']  # 16 ~ 19
    series2["sixthHour"]  = data['sixthHour']  # 20 ~ 23
    population = population.append(series2, ignore_index=True)
    print(simpleLoc)

부산 부산진구 부전2동
부산 기장군 기장읍
부산 해운대구 우2동
부산 부산진구 부전1동
부산 동구 범일2동
부산 해운대구 중1동
부산 기장군 정관읍
부산 해운대구 우1동
부산 남구 대연3동
부산 해운대구 좌2동
부산 사상구 괘법동
부산 수영구 남천1동
부산 해운대구 좌1동
부산 중구 광복동
부산 사하구 하단2동
부산 부산진구 범천1동
부산 부산진구 전포1동
부산 수영구 민락동
부산 중구 중앙동
부산 남구 용호1동
부산 연제구 연산9동
부산 해운대구 반여1동
부산 연제구 연산5동
부산 동래구 온천1동
부산 해운대구 재송1동
부산 사하구 당리동
부산 사상구 감전동
부산 금정구 남산동
부산 사하구 다대1동
부산 사상구 모라1동
부산 동래구 수민동
부산 동래구 안락2동
부산 중구 남포동
부산 수영구 수영동
부산 북구 화명3동
부산 연제구 연산4동
부산 수영구 광안2동
부산 동래구 명륜동
부산 북구 구포1동
부산 동래구 온천3동
부산 사하구 하단1동
부산 북구 덕천2동
부산 남구 대연1동
부산 부산진구 양정1동
부산 동래구 온천2동
부산 영도구 동삼1동
부산 금정구 구서2동
부산 중구 부평동
부산 연제구 거제1동
부산 연제구 연산1동
부산 동구 초량3동
부산 수영구 광안1동
부산 사상구 엄궁동
부산 기장군 장안읍
부산 사하구 장림2동
부산 기장군 일광면
부산 부산진구 초읍동
부산 부산진구 전포2동
부산 동래구 안락1동
부산 금정구 구서1동
부산 수영구 광안3동
부산 부산진구 개금1동
부산 사상구 학장동
부산 수영구 망미1동
부산 강서구 명지1동
부산 남구 대연5동
부산 강서구 대저2동
부산 영도구 남항동
부산 동래구 사직2동
부산 강서구 대저1동
부산 사하구 괴정3동
부산 북구 구포2동
부산 금정구 장전1동
부산 해운대구 반송1동
부산 사하구 괴정1동
부산 해운대구 반송2동
부산 금정구 서3동
부산 사하구 신평1동
부산 해운대구 재송2동
부산 부산진구 가야1동
부산 연제구 연산2동
부산 부산진구 당감1동
부산 동래구 사직1동
부산 금정구 부곡3동
부산 사상구 주례2동


In [8]:
population

,day,dayAvg,fifthHour,firstHour,fourthHour,fri,loc,mon,sat,secondHour,sixthHour,sun,thirdHour,thur,tues,wed,weekend
0,72.35,11079.0,27.26,6.56,25.66,15.75,부산 부산진구 부전2동,13.76,17.09,5.69,16.64,10.56,18.19,14.37,14.21,14.26,27.65
1,67.83,12464.0,25.96,4.29,31.54,13.80,부산 기장군 기장읍,13.94,17.11,6.82,11.16,15.07,20.23,13.39,13.30,13.39,32.17
2,71.58,11525.0,25.81,5.27,25.55,15.25,부산 해운대구 우2동,13.74,16.12,7.16,14.46,12.29,21.75,14.46,14.00,14.13,28.42
3,77.60,7573.0,27.04,3.59,28.24,16.07,부산 부산진구 부전1동,14.67,13.67,6.22,12.35,8.73,22.56,15.57,15.72,15.57,22.40
4,78.63,5224.0,25.51,3.28,29.23,16.38,부산 동구 범일2동,14.80,13.86,6.34,11.62,7.51,24.02,15.72,16.00,15.73,21.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,79.66,963.0,23.15,4.03,27.34,15.99,부산 영도구 봉래2동,14.93,12.36,9.96,10.00,7.98,25.52,16.39,16.30,16.04,20.34
199,75.10,976.0,27.65,4.67,24.18,15.85,부산 사상구 모라3동,14.70,14.42,8.66,12.33,10.48,22.51,15.44,14.47,14.64,24.90
200,76.22,180.0,26.27,4.07,24.92,15.52,부산 동구 수정5동,14.25,13.96,8.07,13.38,9.83,23.29,15.70,15.19,15.56,23.78
201,77.45,3771.0,26.73,4.58,25.50,16.38,부산 금정구 금사회동동,14.72,13.34,9.95,9.52,9.22,23.72,15.40,15.31,15.64,22.55


In [9]:
salesAmount.to_csv("매출액.csv")
population.to_csv("유동인구.csv")